In [5]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

from pyspark.sql import SQLContext

sc1 = SQLContext(sc)

In [2]:
#get user data
userRdd = spark.read.json("D:\\BigDataProject\\yelpDataset\\user.json")

In [6]:
user2 = spark.read.format('csv').options(header='true', inferSchema='true').load("D:\\BigDataProject\\yelp_user.csv")

In [8]:
#save user data in csv
userRdd.repartition(1).write.option("header", "true").csv("user_trend_f.csv")

In [4]:
userRdd.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [2]:
from elasticsearch import helpers, Elasticsearch
import csv
es = Elasticsearch()

In [ ]:
#Load user data into elastic search with index mapping
with open('user_trend_f.csv') as f:
    index_name = 'user_trend_f2'
    doctype = 'user7'
    reader = csv.reader(f)
    headers = []
    index = 0
    es.indices.delete(index=index_name, ignore=[400, 404])
    es.indices.create(index=index_name, ignore=400)
    es.indices.put_mapping(
        index=index_name,
        doc_type=doctype,
        ignore=400,
        body={
            doctype: {
                "properties": {
                    "review_count": {
                        "type": "number",
                    },
                    "yelping_since":{
                        "type":"date",
                        "format": "yyyy-MM-dd"
                    }
                }
            }
        }
    )
    for row in reader:
        try:
            if(index == 0):
                  headers = row
            else:
                row[0] = int(row[0])
                obj = {}
                print(row)
                for i, val in enumerate(row):
                    #print(type(val))
                    obj[headers[i]] = val
                #print(obj)
                  # put document into elastic search
                es.index(index=index_name, doc_type=doctype, body=obj)
                #print(obj)

        except Exception as e:
            print('error: ' + str(e) + ' in' + str(index))
        index = index + 1
    
f.close()